# Agrupamento de Fórmulas Sintéticas com KMeans
Este notebook contém o pipeline completo, desde a vetorização até a exportação final.

**Arquivo de origem:** `csfr_com_formula_unica.xlsx`

**Coluna analisada:** `Formula sintetica`

In [ ]:

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import openpyxl
from docx import Document
from docx.shared import Inches
import datetime
from pathlib import Path

arquivo = "csfr_com_formula_unica.xlsx"
df = pd.read_excel(arquivo, sheet_name='Sheet')
df.head()


In [ ]:

df_validas = df[(df['Formula sintetica'].notna()) & (df['Formula sintetica'] != '-')].copy()
df_validas['Formula sintetica'] = df_validas['Formula sintetica'].astype(str)
formulas_unicas = df_validas['Formula sintetica'].dropna().unique()
df_formulas = pd.DataFrame({'Formula': formulas_unicas})

componentes = ['IGP', 'IPAME', 'IPAMB', 'IPAMQ', 'IPAPQ', 'IPCA', 'TPO']

def extrair_coeficientes_regex(formula):
    coef = dict.fromkeys(componentes, 0.0)
    formula = str(formula).replace(',', '.')
    matches = re.findall(r'([\d.]+)\.([A-Z]+)', formula)
    for valor, indice in matches:
        if indice in coef:
            try:
                coef[indice] = float(valor)
            except ValueError:
                pass
    return [coef[c] for c in componentes]

df_formulas['vetor'] = df_formulas['Formula'].apply(extrair_coeficientes_regex)
X = np.vstack(df_formulas['vetor'].values)
df_formulas.head()


In [ ]:

k_range = range(3, 26)
wcss = []
silhouette_scores = []

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X)
    wcss.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X, labels))

k_otimo = k_range[np.argmax(silhouette_scores)]
print(f"k ótimo encontrado: {k_otimo}")


In [ ]:

fig, ax1 = plt.subplots(figsize=(12, 6))
ax1.set_xlabel("Número de Clusters (k)")
ax1.set_ylabel("WCSS", color='tab:blue')
ax1.plot(k_range, wcss, marker='o', color='tab:blue', label='WCSS')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.grid(True)

for mark_k in [3, 8, 10, 12]:
    ax1.axvline(x=mark_k, color='red', linestyle='--', label=f'k = {mark_k}')

ax2 = ax1.twinx()
ax2.set_ylabel("Silhouette Score", color='tab:green')
ax2.plot(k_range, silhouette_scores, marker='s', color='tab:green', label='Silhouette')
ax2.tick_params(axis='y', labelcolor='tab:green')

plt.title("Comparação WCSS + Silhouette Score (k=3 a 25)")
fig.tight_layout()

grafico_path = "Elbow_Silhouette_Combined.png"
plt.savefig(grafico_path)
plt.show()


In [ ]:

k_escolhido = 8

kmeans_final = KMeans(n_clusters=k_escolhido, random_state=42, n_init=10)
df_formulas['cluster'] = kmeans_final.fit_predict(X)

df_mapeamento_cluster = pd.DataFrame({
    'Formula sintetica': df_formulas['Formula'],
    'cluster': df_formulas['cluster']
})

df_merge = df.merge(df_mapeamento_cluster, on='Formula sintetica', how='left')

wb = openpyxl.Workbook()
ws = wb.active
ws.title = f"Clusters_k{k_escolhido}"

for j, col in enumerate(df_merge.columns, 1):
    ws.cell(row=1, column=j, value=col)

cores = ["FFFFCC", "CCFFCC", "CCCCFF", "FFCCCC", "CCE5FF", "E6CCFF", "FFE699", "D9EAD3", "F4CCCC", "D0E0E3"] * 4

for i, row in enumerate(df_merge.itertuples(index=False), 2):
    cluster_value = getattr(row, 'cluster')
    fill = openpyxl.styles.PatternFill(start_color=cores[int(cluster_value) % len(cores)], fill_type='solid') if not pd.isna(cluster_value) else None

    for j, val in enumerate(row, 1):
        cell = ws.cell(row=i, column=j, value=val)
        if fill and df_merge.columns[j - 1] == 'cluster':
            cell.fill = fill

arquivo_final = "csfr_com_formula_unica_com_clusters_k8.xlsx"
wb.save(arquivo_final)
print(f"Arquivo salvo: {arquivo_final}")


In [ ]:

relatorio = Document()
relatorio.add_heading("Relatório Completo de Análise de Agrupamento", level=1)
relatorio.add_paragraph(f"Data da análise: {datetime.datetime.now().strftime('%d/%m/%Y %H:%M')}")

relatorio.add_heading("1. Introdução", level=2)
relatorio.add_paragraph("Este relatório documenta o processo de agrupamento realizado sobre as fórmulas da coluna 'Formula sintetica'.")

relatorio.add_heading("2. Vetorização", level=2)
relatorio.add_paragraph("Cada fórmula foi convertida em um vetor de 7 dimensões com base nos índices econômicos.")

relatorio.add_heading("3. Agrupamento via KMeans", level=2)
relatorio.add_paragraph("KMeans foi aplicado com k variando de 3 a 25.")

relatorio.add_heading("4. Gráfico WCSS e Silhouette Score", level=2)
if Path(grafico_path).exists():
    relatorio.add_picture(grafico_path, width=Inches(6))
    relatorio.add_paragraph("Gráfico gerado para apoio na escolha do k. Foi escolhido k=8.")
else:
    relatorio.add_paragraph("⚠️ Gráfico não encontrado.")

relatorio.add_heading("5. Exportação", level=2)
relatorio.add_paragraph("A coluna 'cluster' foi adicionada ao arquivo original com coloração.")

relatorio.add_heading("6. Arquivos Gerados", level=2)
relatorio.add_paragraph("- csfr_com_formula_unica_com_clusters_k8.xlsx
- Gráfico WCSS + Silhouette
- Relatório")

relatorio.add_heading("7. Conclusão", level=2)
relatorio.add_paragraph("O agrupamento em 8 clusters foi considerado adequado.")

relatorio_path = "Relatorio_Completo_Agrupamento_FOS_Unico_k8.docx"
relatorio.save(relatorio_path)
print(f"Relatório salvo: {relatorio_path}")
